## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-03-02-04-48 +0000,latimes,Woman charged with murder in connection with a...,https://www.latimes.com/california/story/2025-...
1,2025-10-03-02-03-08 +0000,nyt,Flights Halted at Munich Airport After Drone S...,https://www.nytimes.com/2025/10/02/world/europ...
2,2025-10-03-01-47-06 +0000,nypost,"Trump considering $2,000 tariff ‘dividend’ for...",https://nypost.com/2025/10/02/us-news/trump-co...
3,2025-10-03-01-44-40 +0000,latimes,Orange County woman admits to stealing $2.8 mi...,https://www.latimes.com/california/story/2025-...
4,2025-10-03-01-43-00 +0000,wsj,Bank of Japan Still on Path to More Rate Hikes...,https://www.wsj.com/articles/bank-of-japan-sti...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,35
93,shutdown,21
59,attack,15
58,manchester,15
220,synagogue,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...,117
10,2025-10-03-01-13-09 +0000,nyt,Trump Escalates Attacks on Democrats in a Gove...,https://www.nytimes.com/2025/10/02/us/politics...,87
54,2025-10-02-22-55-39 +0000,nypost,Trump plans for ‘thousands’ of federal firings...,https://nypost.com/2025/10/02/us-news/trump-sh...,84
237,2025-10-02-13-17-00 +0000,wsj,Office of Management and Budget Director Russe...,https://www.wsj.com/politics/policy/russell-vo...,79
63,2025-10-02-22-12-02 +0000,nypost,Israeli officials rip ‘weak’ UK government for...,https://nypost.com/2025/10/02/world-news/israe...,78


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,117,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...
274,75,2025-10-02-10-09-41 +0000,cbc,U.K. police say deadly rampage at Manchester s...,https://www.cbc.ca/news/world/uk-synagogue-inc...
86,48,2025-10-02-21-25-48 +0000,nypost,Ukraine pitches plan to help US sell American ...,https://nypost.com/2025/10/02/world-news/ukrai...
21,44,2025-10-03-00-45-00 +0000,wsj,A multibillion-dollar deal to send Nvidia’s AI...,https://www.wsj.com/politics/policy/nvidia-tru...
231,38,2025-10-02-13-39-46 +0000,wapo,"Israel intercepts Gaza aid flotilla, detains T...",https://www.washingtonpost.com/world/2025/10/0...
9,36,2025-10-03-01-22-00 +0000,wsj,Senators on both sides of the aisle pressed th...,https://www.wsj.com/politics/national-security...
280,36,2025-10-02-10-00-00 +0000,nypost,NYC’s digital members-only clubs are harnessin...,https://nypost.com/2025/10/02/business/soho-ho...
16,33,2025-10-03-01-00-00 +0000,wsj,"Inside the White House, concerns are growing t...",https://www.wsj.com/politics/policy/trump-gove...
315,32,2025-10-02-05-01-25 +0000,nypost,UK university student dies by suicide 3 months...,https://nypost.com/2025/10/02/world-news/scotl...
237,32,2025-10-02-13-17-00 +0000,wsj,Office of Management and Budget Director Russe...,https://www.wsj.com/politics/policy/russell-vo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
